In [ ]:
# Installer dépendances si besoin
!pip install mediapipe moviepy


In [ ]:
import cv2
import numpy as np
from moviepy.editor import *
from PIL import Image
import mediapipe as mp
from google.colab import files
import os

mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def align_face_no_zoom(image_rgb, output_size=(1280,720)):
    h, w, _ = image_rgb.shape
    img = image_rgb.astype(np.uint8)
    res = mp_face_mesh.process(img)
    
    left_eye, right_eye = None, None
    if res and res.multi_face_landmarks:
        lm = res.multi_face_landmarks[0].landmark
        left_idxs = [33, 133]
        right_idxs = [362, 263]
        left_eye = np.mean([(lm[i].x * w, lm[i].y * h) for i in left_idxs], axis=0)
        right_eye = np.mean([(lm[i].x * w, lm[i].y * h) for i in right_idxs], axis=0)
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        if len(faces) == 0:
            return None
        x, y, wb, hb = max(faces, key=lambda f: f[2]*f[3])
        cx = x + wb/2
        cy = y + hb/3.0
        left_eye = np.array([cx - wb*0.18, cy])
        right_eye = np.array([cx + wb*0.18, cy])
    left_eye = np.array(left_eye, dtype=np.float32)
    right_eye = np.array(right_eye, dtype=np.float32)

    eyes_center = (left_eye + right_eye) / 2.0
    dx, dy = right_eye[0] - left_eye[0], right_eye[1] - left_eye[1]
    angle = np.degrees(np.arctan2(dy, dx))
    eye_dist = np.hypot(dx, dy)
    if eye_dist < 1.0:
        return None

    crop_width = eye_dist * 4.0
    crop_height = crop_width * (output_size[1] / output_size[0])
    center_x, center_y = eyes_center
    x1 = int(center_x - crop_width / 2)
    y1 = int(center_y - crop_height / 2)
    x2 = int(center_x + crop_width / 2)
    y2 = int(center_y + crop_height / 2)

    if x1 < 0 or y1 < 0 or x2 > w or y2 > h:
        M = cv2.getRotationMatrix2D((center_x, center_y), angle, 1.0)
        rotated_full = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
        final_img = cv2.resize(rotated_full, output_size, interpolation=cv2.INTER_AREA)
        return final_img

    cropped = img[y1:y2, x1:x2]
    crop_h, crop_w = cropped.shape[:2]
    M = cv2.getRotationMatrix2D((crop_w/2, crop_h/2), angle, 1.0)
    rotated = cv2.warpAffine(cropped, M, (crop_w, crop_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    final_img = cv2.resize(rotated, output_size, interpolation=cv2.INTER_AREA)
    return final_img

def create_face_movie(files, duration_per_face=4.0, fade_duration=2.5):
    clips = []
    for f in files:
        try:
            pil = Image.open(f.name).convert('RGB')
            img = np.array(pil)
        except:
            continue
        aligned = align_face_no_zoom(img)
        if aligned is None:
            continue
        clip = ImageClip(aligned).set_duration(duration_per_face)
        clips.append(clip)
    if not clips:
        return "Aucune image utilisable", None
    final = clips[0]
    for next_clip in clips[1:]:
        final = concatenate_videoclips([final.crossfadeout(fade_duration), next_clip.crossfadein(fade_duration)], method='compose')
    out_path = 'face_movie_output.mp4'
    final.write_videofile(out_path, fps=24, codec='libx264', audio=False)
    return f"Vidéo générée avec {len(clips)} images", out_path


In [ ]:
# Upload images
uploaded_files = files.upload()
status, video_path = create_face_movie(uploaded_files.values())
print(status)
if video_path:
    from google.colab import files as gfiles
    gfiles.download(video_path)
